# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Import for adding the marker layer on second map
import json 
import pprint


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data_file = "Resources/Cities.csv"
cities_data = pd.read_csv(cities_data_file)

cities_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,71.01,60,41,5.99,AU,1615525401
1,Sur,22.5667,59.5289,85.30,35,0,2.77,OM,1615525364
2,Saskylakh,71.9167,114.0833,-7.92,88,100,16.84,RU,1615525552
3,Mataura,-46.1927,168.8643,64.00,67,1,7.00,NZ,1615525552
4,Albany,42.6001,-73.9662,60.80,55,75,13.80,US,1615525553


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
city_coordinates = cities_data[["Lat", "Lng"]]
cities_humidity = cities_data["Humidity"]

fig = gmaps.figure(center=(50.0, -35.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(city_coordinates, weights=cities_humidity,
                                  dissipating=False, max_intensity=10, 
                                  point_radius=2)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
perfect_temperature = cities_data.loc[(cities_data["Max Temp"] >70) &
                                      (cities_data["Max Temp"] < 80) &
                                      (cities_data["Wind Speed"] < 10) &
                                      (cities_data["Cloudiness"] == 0), :] 

perfect_temperature = perfect_temperature.dropna(how='any')  
perfect_temperature

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,Ambovombe,-25.1667,46.0833,78.78,74,0,7.36,MG,1615525561
36,Vila Velha,-20.3297,-40.2925,77.00,88,0,3.44,BR,1615525327
57,Maceió,-9.6658,-35.7353,75.20,94,0,2.30,BR,1615525462
111,Pisco,-13.7000,-76.2167,73.40,78,0,3.44,PE,1615525672
171,Jizan,17.3333,42.6667,77.00,83,0,2.30,SA,1615525756
178,Umluj,25.0213,37.2685,78.30,47,0,3.85,SA,1615525781
206,Salina Cruz,16.1667,-95.2000,77.14,76,0,6.35,MX,1615525846
339,Pilar,-9.5972,-35.9567,75.20,94,0,2.30,BR,1615526137
396,Bandarbeyla,9.4942,50.8122,79.12,64,0,9.26,SO,1615526260


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = perfect_temperature.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
31,Ambovombe,MG,-25.1667,46.0833,
36,Vila Velha,BR,-20.3297,-40.2925,
57,Maceió,BR,-9.6658,-35.7353,
111,Pisco,PE,-13.7000,-76.2167,
171,Jizan,SA,17.3333,42.6667,
178,Umluj,SA,25.0213,37.2685,
206,Salina Cruz,MX,16.1667,-95.2000,
339,Pilar,BR,-9.5972,-35.9567,
396,Bandarbeyla,SO,9.4942,50.8122,


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
#   pprint.pprint(response)
#   break
   
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No hotel")
        
    print("------------")


Retrieving Results for Index 31: Ambovombe.
Closest hotel in Ambovombe is Hôtel Étoile du Sud.
------------
Retrieving Results for Index 36: Vila Velha.
Closest hotel in Vila Velha is Golden Tulip Porto Vitória.
------------
Retrieving Results for Index 57: Maceió.
Closest hotel in Maceió is Jatiúca Hotel & Resort.
------------
Retrieving Results for Index 111: Pisco.
Closest hotel in Pisco is PiscoMar Peru.
------------
Retrieving Results for Index 171: Jizan.
No hotel
------------
Retrieving Results for Index 178: Umluj.
Closest hotel in Umluj is HP Red Sea Hotel.
------------
Retrieving Results for Index 206: Salina Cruz.
Closest hotel in Salina Cruz is Hotel Mision San José.
------------
Retrieving Results for Index 339: Pilar.
No hotel
------------
Retrieving Results for Index 396: Bandarbeyla.
Closest hotel in Bandarbeyla is JABIR HOTEL.
------------


In [7]:
 hotel_df

,City,Country,Lat,Lng,Hotel Name
31,Ambovombe,MG,-25.1667,46.0833,Hôtel Étoile du Sud
36,Vila Velha,BR,-20.3297,-40.2925,Golden Tulip Porto Vitória
57,Maceió,BR,-9.6658,-35.7353,Jatiúca Hotel & Resort
111,Pisco,PE,-13.7000,-76.2167,PiscoMar Peru
171,Jizan,SA,17.3333,42.6667,
178,Umluj,SA,25.0213,37.2685,HP Red Sea Hotel
206,Salina Cruz,MX,16.1667,-95.2000,Hotel Mision San José
339,Pilar,BR,-9.5972,-35.9567,
396,Bandarbeyla,SO,9.4942,50.8122,JABIR HOTEL


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))